In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import os
os.environ['CUDA_VISIBLE_DEVICES'] = input('Input utilize gpu-id (cpu:-1): ')
import sys
sys.path.append('utils/')

In [ ]:
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_integer('seq_length', 10,
                            """seq_length""")
tf.app.flags.DEFINE_integer('seq_start', 4,
                            """ start of seq generation""")
tf.app.flags.DEFINE_integer('batch_size', 1,
                            """batch size for training""")
tf.app.flags.DEFINE_integer('height', 128,
                            """height of input data""")
tf.app.flags.DEFINE_integer('width', 128,
                            """width of input data""")
tf.app.flags.DEFINE_string('warping', 'bilinear',
                            """method of warping""")
tf.app.flags.DEFINE_integer('gating_num', 3,
                            """# of experts""")
tf.app.flags.DEFINE_string('activation', 'softmax',
                            """gating activation""")
tf.app.flags.DEFINE_boolean('restore_all', True, 
                            """whether restore all or not """)
tf.app.flags.DEFINE_boolean('restore_gating', True, 
                            """whether restore gating or not """)
tf.app.flags.DEFINE_integer('h_conv_ksize', 1,
                            """kernel size of H""")
tf.app.flags.DEFINE_boolean('trainable_last', True, 
                            """train backtopixel conv(H) of MIM or not""")
tf.app.flags.DEFINE_string('f', '', 'kernel')

In [ ]:
import layer_def as ld
from warp import get_pixel_value
from warp import bilinear_warp as tf_warp
import w_initializer

In [ ]:
from Expert import network_shift
from Expert import MIM
network_rot = MIM
network_grow = MIM
from Gating import network_gate

In [ ]:
network_gate = tf.make_template('gate', network_gate)
network_shift = tf.make_template('network_convdeconv', network_shift)

In [ ]:
def test(model1, model2, model3, model4, model5, test_data):
    """
    model1: model path for translation
    model2: model path for rotation
    model3: model path for growth/decay
    model4: model path for gating
    model5: model path for the whole STMoE model
    train_data: np.array for test
    """
    x= tf.placeholder(tf.float32, [FLAGS.batch_size, FLAGS.height, FLAGS.width, FLAGS.seq_length])
    x_g = []
    weights = []
    f_g_s = []
    x_g_s, x_g_r, x_g_g = [], [], []
    
    hidden_state_1, hidden_state_diff_1, cell_state_1, cell_state_diff_1, st_memory_1 = [], [], [], [], []
    hidden_state_2, hidden_state_diff_2, cell_state_2, cell_state_diff_2, st_memory_2 = [], [], [], [], []       
    
    for i in range(FLAGS.seq_start - 1):
            with tf.variable_scope('expert2'):
                inputs = x[:, :, :, i]
                inputs = inputs[:, :, :, tf.newaxis]
                x_generate_r, hidden_state_1, hidden_state_diff_1, cell_state_1, cell_state_diff_1, st_memory_1 =                     network_grow(inputs, i, hidden_state_1, hidden_state_diff_1, cell_state_1, cell_state_diff_1, st_memory_1, 3, [8, 8, 8], (3, 3), FLAGS.h_conv_ksize, stride=1, tln = True, trainable_last = FLAGS.trainable_last)
                
            with tf.variable_scope('expert3'):
                inputs = x[:, :, :, i]
                inputs = inputs[:, :, :, tf.newaxis]
                x_generate_g, hidden_state_2, hidden_state_diff_2, cell_state_2, cell_state_diff_2, st_memory_2 =                     network_grow(inputs, i, hidden_state_2, hidden_state_diff_2, cell_state_2, cell_state_diff_2, st_memory_2, 3, [8, 8, 8], (3, 3), FLAGS.h_conv_ksize, stride=1, tln = True, trainable_last = FLAGS.trainable_last)      
    
    # predict recursively
    for i in range(FLAGS.seq_length - FLAGS.seq_start):
        print('frame_{}'.format(i))
        if i == 0:
            with tf.variable_scope('expert1'):
                f_generate = network_shift(x[:, :, :, i:i+FLAGS.seq_start])
                last_x = x[:, :, :, FLAGS.seq_start - 1]
                x_generate_s = tf_warp(last_x[:, :, :, tf.newaxis], f_generate, FLAGS.height, FLAGS.width)
                x_generate_s = tf.reshape(x_generate_s[:, :, :, 0], [FLAGS.batch_size, FLAGS.height, FLAGS.width, 1])
                x_g_s.append(x_generate_s)

            with tf.variable_scope('expert2'):
                inputs = x[:, :, :,  FLAGS.seq_start - 1]
                inputs = inputs[:, :, :, tf.newaxis]
                x_generate_r, hidden_state_1, hidden_state_diff_1, cell_state_1, cell_state_diff_1, st_memory_1 =                      network_grow(inputs, i + FLAGS.seq_start, hidden_state_1, hidden_state_diff_1, cell_state_1, cell_state_diff_1, st_memory_1, 3, [8, 8, 8], (3, 3), FLAGS.h_conv_ksize, stride=1, tln = True, trainable_last = FLAGS.trainable_last)
                x_g_r.append(x_generate_r)

            with tf.variable_scope('expert3'):
                inputs = x[:, :, :,  FLAGS.seq_start - 1]
                inputs = inputs[:, :, :, tf.newaxis]
                x_generate_g, hidden_state_2, hidden_state_diff_2, cell_state_2, cell_state_diff_2, st_memory_2 =                      network_grow(inputs, i + FLAGS.seq_start, hidden_state_2, hidden_state_diff_2, cell_state_2, cell_state_diff_2, st_memory_2, 3, [8, 8, 8], (3, 3), FLAGS.h_conv_ksize, stride=1, tln = True, trainable_last = FLAGS.trainable_last)
                x_g_g.append(x_generate_g)             

            with tf.variable_scope('gating_network'):
                weight = network_gate(x[:, :, :, i:i+FLAGS.seq_start], FLAGS.gating_num, moe='moe1')
                x_sr = tf.concat([x_generate_s, x_generate_r, x_generate_g], axis = -1)
                x_generate = weight * x_sr
                x_generate = tf.reduce_sum(x_generate, axis=-1)
                x_g.append(x_generate)
                weights.append(weight)

        else:
            x_gen = tf.stack(x_g)
            print(x_gen.shape)
            x_gen = tf.transpose(x_gen, [1, 2, 3, 0])

            if i < FLAGS.seq_start:
                x_input = tf.concat([x[:, :, :, i:FLAGS.seq_start], x_gen[:,:, :, :i]], axis = 3)
            else:
                x_input = x_gen[:, :, :, i - FLAGS.seq_start:i]

            with tf.variable_scope('expert1'):
                f_generate = network_shift(x_input)
                last_x = x_g[-1]
                x_generate_s = tf_warp(last_x[:, :, :, tf.newaxis], f_generate,  FLAGS.height, FLAGS.width)
                x_generate_s = tf.reshape(x_generate_s[:, :, :, 0], [FLAGS.batch_size, FLAGS.height, FLAGS.width, 1])
                x_g_s.append(x_generate_s)

            with tf.variable_scope('expert2'):
                inputs = x_g[-1]
                inputs = inputs[:, :, :, tf.newaxis]
                x_generate_r, hidden_state_1, hidden_state_diff_1, cell_state_1, cell_state_diff_1, st_memory_1 =                      network_grow(inputs, i + FLAGS.seq_start, hidden_state_1, hidden_state_diff_1, cell_state_1, cell_state_diff_1, st_memory_1, 3, [8, 8, 8], (3, 3), FLAGS.h_conv_ksize, stride=1, tln = True, trainable_last = FLAGS.trainable_last)
                x_g_r.append(x_generate_r)

            with tf.variable_scope('expert3'):
                inputs = x_g[-1]
                inputs = inputs[:, :, :, tf.newaxis]
                x_generate_g, hidden_state_2, hidden_state_diff_2, cell_state_2, cell_state_diff_2, st_memory_2 =                      network_grow(inputs, i + FLAGS.seq_start, hidden_state_2, hidden_state_diff_2, cell_state_2, cell_state_diff_2, st_memory_2, 3, [8, 8, 8], (3, 3), FLAGS.h_conv_ksize, stride=1, tln = True, trainable_last = FLAGS.trainable_last)
                x_g_g.append(x_generate_g) 

            with tf.variable_scope('gating_network'):
                weight = network_gate(x_input, FLAGS.gating_num, moe='moe1')
                x_sr = tf.concat([x_generate_s, x_generate_r, x_generate_g], axis = -1)
                x_generate = weight*x_sr
                x_generate = tf.reduce_sum(x_generate, axis=-1)
                x_g.append(x_generate) 
                weights.append(weight)
                
    x_g = tf.stack(x_g)
    x_g = tf.transpose(x_g, [1, 2, 3, 0])

    weights = tf.stack(weights)
    weights = tf.transpose(weights, [1, 0, 2, 3, 4])
    
    # build a saver 
    expert1_varlist = {v.op.name.lstrip("expert1/"): v 
          for v in tf.get_collection(tf.GraphKeys.VARIABLES, scope="expert1/")} 

    expert1_saver = tf.train.Saver(var_list=expert1_varlist) 
    
    expert2_varlist = {v.op.name[8:]: v 
          for v in tf.get_collection(tf.GraphKeys.VARIABLES, scope="expert2/")} 
    expert2_saver = tf.train.Saver(var_list=expert2_varlist) 
    
    expert3_varlist = {v.op.name[8:]: v 
          for v in tf.get_collection(tf.GraphKeys.VARIABLES, scope="expert3/")} 
    expert3_saver = tf.train.Saver(var_list=expert3_varlist)   
    
    # build a gating saver
    gating_varlist = {v.name.lstrip("gating_network/"): v 
           for v in tf.get_collection(tf.GraphKeys.VARIABLES, scope="gating_network/")} 
    gating_saver = tf.train.Saver(var_list=gating_varlist, max_to_keep=100)    
    
    # MSE loss
    MSE = tf.losses.mean_squared_error(x[:, :, :, FLAGS.seq_start:], x_g[:, :, :, :])
    
    # SSIM
    SSIM = tf.image.ssim(x[:, :, :, FLAGS.seq_start:], x_g[:, :, :, :], 1)

    # List of all varables
    variables = tf.global_variables()

    # strat rinning operations on Graph
    sess = tf.Session()
    init = tf.global_variables_initializer()
    
    print('init netwrok from scratch....')
    sess.run(init)
                         
    expert1_saver.restore(sess, model1)

    expert2_saver.restore(sess, model2)
    
    expert3_saver.restore(sess, model3)
    
    # restore gating saver
    if FLAGS.restore_gating:
        gating_saver.restore(sess, model4)
    
    # restore all saver
    all_saver =  tf.train.Saver(max_to_keep=100)
    
    if FLAGS.restore_all:
        all_saver.restore(sess, model5)
            
    Loss_MSE, Loss_MSE_v  = [], []
    all_Loss, all_Loss_v = [], []
    Pred, Pred_s, Pred_r, Pred_g, Pred_t, W_pre, Loss, SSIM_test  = [], [], [], [], [], [], [], []
    # test
    for i, da in enumerate(test_data):
        batch_x = da[np.newaxis, :, :, :]
        batch_x = batch_x.transpose(0, 2, 3, 1)
        pred, pred_s, pred_r, pred_g, w_pre, mse_test, ssim_test = sess.run([x_g, x_g_s, x_g_r, x_g_g, weights, MSE, SSIM],
                                                                                     feed_dict = {x:batch_x}) 
        
        Pred.append(pred)
        Pred_s.append(pred_s)
        Pred_r.append(pred_r)
        Pred_g.append(pred_g)      
        W_pre.append(w_pre)
        Loss.append(mse_test)
        SSIM_test.append(ssim_test)
        
        
    return Pred, Pred_s, Pred_r, Pred_g, W_pre, Loss, SSIM_test

In [ ]:
def main():
    model1 = './models/STMoE-1_trans'
    model2 = './models/STMoE-1_rot'
    model3 = './models/STMoE-1_grow' 
    model4 = './models/STMoE-1_gating'
    model5 = './models/STMoE-1_all'


    test_data = np.load('./data/test_data.npy')
    max_intensity = 255
        
    test_data = test_data / max_intensity
    Pred, Pred_s, Pred_r, Pred_g, W_pre, Loss, SSIM_test = test(model1, model2, model3, model4, model5, test_data)
    
    return Pred, Pred_s, Pred_r, Pred_g, W_pre, Loss, SSIM_test, test_data

In [ ]:
if __name__ == '__main__':
    Pred, Pred_s, Pred_r, Pred_g, W_pre, Loss, SSIM_test, test_data = main()

In [ ]:
%matplotlib inline

## Loss

In [ ]:
Loss = np.array(Loss)*255**2
Loss.mean()

In [ ]:
SSIM = np.array(SSIM_test)
SSIM.mean()

## Visualization

In [ ]:
Pred = np.array(Pred)*255
Pred = Pred[:, 0, :, :, :]
test_data = test_data*255

In [ ]:
num = 79
vmx = 255

In [ ]:
## pred vs gt

In [ ]:
for i in range(FLAGS.seq_length):   
    if i >= FLAGS.seq_start:
        plt.subplot(1, 2, 2)
        plt.imshow(Pred[num, :, :, i-FLAGS.seq_start], vmin = 0, vmax = vmx, cmap = 'jet') 
        plt.title('est_{}'.format(i))
        plt.colorbar()
    plt.subplot(1, 2, 1)
    plt.imshow(test_data[num, i, :, :], vmin = 0, vmax = vmx, cmap = 'jet')   
    plt.title('gt_{}'.format(i))
    plt.colorbar()
    plt.show()

In [ ]:
plt.figure(figsize=(20, 20))
for i in range(FLAGS.seq_length):   
    if i >= FLAGS.seq_start:
        plt.subplot(1, 10, i+1)
        plt.imshow(Pred[num, :, :, i-FLAGS.seq_start], vmin = 0, vmax = vmx, cmap = 'jet') 
        plt.title('est_{}'.format(i))
plt.show()

## pred and contribution of each experts

In [ ]:
W_pre = np.array(W_pre)
W_pre.shape
W_pre = W_pre[:, 0, :, :, :, :]

In [ ]:
nums =  [79]
k = ['translation', 'rot', 'g/d']
axes = []
fig=plt.figure(figsize=(12, 9))
for i, n in enumerate(nums):
    for j in range(3):
        axes.append(fig.add_subplot(4, 4, ((4*i+j+1))))
        plt.imshow(Pred[n, :, :, 1], cmap ='jet', vmin=0, vmax=vmx)
        plt.imshow(W_pre[n, 1, :, :, j], alpha=0.4, vmax=0.8)
        plt.title(k[j])